# 5.3 Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the **Auto** data set.

In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

In [2]:
auto = pd.read_csv('../data/Auto.csv',
                  na_values='?')\
         .dropna()\
         .reset_index()
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         392 non-null    int64  
 1   mpg           392 non-null    float64
 2   cylinders     392 non-null    int64  
 3   displacement  392 non-null    float64
 4   horsepower    392 non-null    float64
 5   weight        392 non-null    int64  
 6   acceleration  392 non-null    float64
 7   year          392 non-null    int64  
 8   origin        392 non-null    int64  
 9   name          392 non-null    object 
dtypes: float64(4), int64(5), object(1)
memory usage: 30.8+ KB


We begin by using the `DataFrame`'s `sample()` method to split the set of observations into two halves, by selecting a random subset of 196 observations out of the original 392 observations.

To reproduce the result, we can specify the `random_state` option in the `sample()` method.

In [3]:
train_auto = auto.sample(196, random_state=1)
test_auto = auto[~auto.isin(train_auto)].dropna(how='all')

We then use the `subset` option in `smf.ols()` to fit a linear regression using only the observations corresponding to the training set.

In [4]:
lm_fit_auto = smf.ols('mpg~horsepower',
                     data=auto,
                     subset=train_auto.index).fit()
lm_fit_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Fri, 08 Jan 2021   Prob (F-statistic):           1.28e-42
Time:                        21:33:15   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.000      38.316      42.352
horsepower    -0.1596      0.009    -17.788      0.000      -0.177      -0.142
==============================================================================
Omnibus:                        8.393   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.015   Jarque-Bera (JB):                8.787
Skew:                           0.516   Prob(JB):                       0.0124
Kurtosis:                       2.899   Cond. No.                         328.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We now use the models's `predict()` method to estimate the response and calculate the MSE of the 196 observations in the validation set.

In [5]:
pred_auto = lm_fit_auto.predict(test_auto)
((pred_auto - test_auto['mpg'])**2).mean()

23.36190289258723

Therefore, the estimated test MSE for the linear regression fit is 23.36. We can use the `np.power()` function to estimate the test error for the quadratic and cubic regressions.

In [6]:
lm_fit2_auto = smf.ols('mpg~horsepower+np.power(horsepower, 2)',
                      data=auto,
                      subset=train_auto.index).fit()
pred2_auto = lm_fit2_auto.predict(test_auto)
((pred2_auto - test_auto['mpg'])**2).mean()

20.252690858350064

In [7]:
lm_fit3_auto = smf.ols('mpg~horsepower+np.power(horsepower, 2)+np.power(horsepower, 3)',
                      data=auto,
                      subset=train_auto.index).fit()
pred3_auto = lm_fit3_auto.predict(test_auto)
((pred3_auto - test_auto['mpg'])**2).mean()

20.32560936589255

These error rates are 20.25 and 20.33 respectively. If we choose a diffrent training set instead, then we will obtain somewhat different errors on the validation set.

In [8]:
train_auto = auto.sample(196, random_state=2)
test_auto = auto[~auto.isin(train_auto)].dropna(how='all')

In [9]:
lm_fit_auto = smf.ols('mpg~horsepower',
                     data=auto,
                     subset=train_auto.index).fit()
pred_auto = lm_fit_auto.predict(test_auto)
((pred_auto - test_auto['mpg'])**2).mean()

25.10853905288965

In [10]:
lm_fit2_auto = smf.ols('mpg~horsepower+np.power(horsepower, 2)',
                      data=auto,
                      subset=train_auto.index).fit()
pred2_auto = lm_fit2_auto.predict(test_auto)
((pred2_auto - test_auto['mpg'])**2).mean()

19.722533470490276

In [11]:
lm_fit3_auto = smf.ols('mpg~horsepower+np.power(horsepower, 2)+np.power(horsepower, 3)',
                      data=auto,
                      subset=train_auto.index).fit()
pred3_auto = lm_fit3_auto.predict(test_auto)
((pred3_auto - test_auto['mpg'])**2).mean()

19.921367860022265

Using this split of the observations into a training set and a validation set, we find that the validation set error rates for the models with linear, quadratic, and cubic terms are 25.11, 19.72, and 19.92, respectively.

## 5.3.2 Leave-One-Out Cross-Validation

The LOOCV estimate can be automatically computed for any generalized linear model using the `LeaveOneOut()` and `KFold()` functions from the sub-module `model_selection` in the `scikit-learn` package.

In [12]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
import  sklearn.linear_model as sk_lm

In [13]:
lm = sk_lm.LinearRegression()

X_train = train_auto['horsepower'].values.reshape(-1,1)
y_train = train_auto['mpg']
X_test = test_auto['horsepower'].values.reshape(-1,1)
y_test = test_auto['mpg']

In [14]:
model = lm.fit(X_train, y_train)

loo = LeaveOneOut()
X = auto['horsepower'].values.reshape(-1,1)
y = auto['mpg'].values.reshape(-1,1)

scores = cross_val_score(lm, X, y,
                         scoring='neg_mean_squared_error',
                         cv=loo,
                         n_jobs=1)
print(f'Folds: {len(scores)}'
      f'\nMSE: {np.mean(np.abs(scores))}' # since the scoring function returns a negative MSE.
      f'\nSTD: {np.std(scores)}')

Folds: 392
MSE: 24.231513517929226
STD: 36.79731503640535


We can see that our cross-validation estimate for the test error is approximately 24.23.

We can repeat this procedure for increasingly complex polynomial fits. In this case, we'll do it in a *for loop* to iteratively fit polynomial regressions for polynomials of order `i=1` to `i=5`, and compute the associated cross-validation error.

In [16]:
from sklearn.preprocessing import PolynomialFeatures

In [21]:
for i in range(1, 6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(lm, X_current, y,
                            scoring='neg_mean_squared_error',
                            cv=loo,
                            n_jobs=1)
    print(f'Degree-{i} polynomial\n'
          f'MSE: {np.mean(np.abs(scores))}\n'
          f'STD: {np.std(scores)}\n'
          '=================================')    

Degree-1 polynomial
MSE: 24.23151351792922
STD: 36.797315036405344
Degree-2 polynomial
MSE: 19.248213124489748
STD: 34.99844615178233
Degree-3 polynomial
MSE: 19.33498406406717
STD: 35.76513567797019
Degree-4 polynomial
MSE: 19.424430308775744
STD: 35.683352759230665
Degree-5 polynomial
MSE: 19.0332148474514
STD: 35.317311409482386


We can see a sharp drop in the estimated test MSE between the linear and quafratic fits, but then no clear improvement from using higher-order polynomials.

## 5.3.3 k-Fold Cross-Validation
The `KFold()` function can be used to implement *k*-fold CV. Below we use `k=10`, a common choice for *k*, on the **Auto** data set. We will also obtain the polynomial fits of orders one to ten.

In [24]:
crossvalidation = KFold(n_splits=10, random_state=1, shuffle=True)

for i in range(1, 11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    scores = cross_val_score(lm, X_current, y,
                             scoring='neg_mean_squared_error',
                             cv=crossvalidation,
                             n_jobs=1)
    print(f'Degree-{i} polynomial\n'
          f'MSE: {np.mean(np.abs(scores))}\n'
          f'STD: {np.std(scores)}\n'
          '====================================')

Degree-1 polynomial
MSE: 24.097675731883058
STD: 4.818054666704995
Degree-2 polynomial
MSE: 19.178889864889662
STD: 5.126393446517313
Degree-3 polynomial
MSE: 19.213859523704357
STD: 5.143687485487353
Degree-4 polynomial
MSE: 19.21280701911446
STD: 4.926661024804993
Degree-5 polynomial
MSE: 18.75797963141857
STD: 4.70323502370233
Degree-6 polynomial
MSE: 18.63518869983694
STD: 4.509539871936917
Degree-7 polynomial
MSE: 18.82096682619666
STD: 4.564907863618866
Degree-8 polynomial
MSE: 18.97573372824936
STD: 4.7117057342608595
Degree-9 polynomial
MSE: 18.93746190765147
STD: 4.869598846436348
Degree-10 polynomial
MSE: 18.79001024796583
STD: 4.840922846529729


We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## 5.3.4 The Bootstrap
We illustrate the use of the bootstrap in the simple example of Section 5.2, as well as on an example involving estimating the accuracy of the linear regression model on the **Auto** data set.

### Estimating the Accuracy of a Statistic of Interest
One of the greate advantages of the bootstrap approach is that it can be applied in almost all situations. 

In this setting, we use the **Portfolio** data set.

In [28]:
portfolio = pd.read_csv('../data/Portfolio.csv',
                        na_values='?',
                        index_col=0)
portfolio.head()

,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


Then we first define a `alpha()` function, which takes as input the `(X, Y)` data to estimate $\alpha$.

In [135]:
def alpha(X, Y):
    result = (np.var(Y) - np.cov(X, Y))/(np.var(X) + np.var(Y) - 2*np.cov(X, Y))
    return result[0,1]

In [136]:
X = portfolio.X[0:100]
Y = portfolio.Y[0:100]
alpha(X, Y)

0.5766511516104116

We can see that the estimated $\alpha$ using all 100 observations is 0.57665115.

The next command uses the `sample()` method to randomly select 100 observations from the range 1 to 100, with replacement. This is equivalent to constructing a new bootstrap data set and recomputing $\hat{\alpha}$based on the new data set.

In [67]:
portfolio_sample = portfolio.sample(frac=1, replace=True)
X_sample = portfolio_sample.X[0:100]
Y_sample = portfolio_sample.Y[0:100]
alpha(X_sample, Y_sample)

array([[1.04481519, 0.61098518],
       [0.61098518, 0.04646865]])

We can then implement a bootstrap analysis by performing this command many times, recording all of the corresponding estimates for $\alpha$, and computing the resulting standard deviation. Below we produce 1,000 bootstrap estimates for $\alpha$:

In [145]:
def boot(df, fun, n, *, x='X', y='Y'):
    tresult = []
    for i in range(0, n):
        dfsample = df.sample(frac=1, replace=True)
        X = dfsample[x]
        Y = dfsample[y]
        result = fun(X, Y)
        tresult.append(result)
    estimate = sum(tresult)/n
    std_est = np.std(tresult, axis=0)
    print('Bootstrap Statistics:\n'
         f'Estimate: {estimate}\n'
         f'STD: {std_est}\n')

In [147]:
boot(portfolio, alpha, 1000)

Bootstrap Statistics:
Estimate: 0.5825695998321582
STD: 0.09024505084391195



The final result shows that using the original data, $\hat{\alpha} = 0.582$, and that the bootstrap estimate for $SE(\hat{\alpha})$ is 0.09.

### Estimating the Accuracy of a Linear Regression Model
The bootstrap approach can be used to assess the variability of the coefficient estimates and predictions from a statistical learning method. Here we use the bootstrap approach in order to assess the variability of the estimates for $\beta_0$ and $\beta_1$, the intercept and slope terms for the linear regression model that uses **horsepower** to predict **mpg** in the **Auto** data set.

We first define a simple function, `coef()`, which takes in the `(X, Y)` parameters and returns the intercept and slope estimates for the linear regression model.

In [128]:
def coef(X, Y):
    lm = sk_lm.LinearRegression()
    model = lm.fit(X.values.reshape(-1,1), Y)
    return np.append(model.intercept_, model.coef_)

In [129]:
coef(auto.horsepower, 
     auto.mpg)

array([39.93586102, -0.15784473])

In [130]:
auto_sample = auto.sample(frac=1, replace=True)
coef(auto_sample.horsepower,
     auto_sample.mpg)

array([41.20546725, -0.16629016])

In [107]:
auto_sample = auto.sample(frac=1, replace=True)
coef(auto_sample.horsepower.values.reshape(-1,1),
     auto_sample.mpg.values.reshape(-1,1))

array([41.09773013, -0.1672969 ])

Next, we use the `boot()` function to compute the standard error of 1,000 bootstrap estimates for the intercept and slope terms.

In [149]:
boot(auto, coef, 1000, x='horsepower', y='mpg')

Bootstrap Statistics:
Estimate: [39.971905   -0.15811471]
STD: [0.8502544  0.00739021]



This indicates that the bootstrap estimates for $SE(\hat{\beta}_0$) is 0.85, and that the bootstrap estimate for $SE(\hat{\beta}_1)$ is 0.0074.

The standard errors for the regression coefficients in a linear model can be computed using the standard formulas. These can be obtained using the `statsmodels` library as follows:

In [151]:
lm_fit = smf.ols('mpg~horsepower',
                data=auto).fit()
lm_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Fri, 08 Jan 2021   Prob (F-statistic):           7.03e-81
Time:                        23:39:33   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
==============================================================================
Omnibus:                       16.432   Durbin-Watson:                   0.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.305
Skew:                           0.492   Prob(JB):                     0.000175
Kurtosis:                       3.299   Cond. No.                         322.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Below we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model to the data. Since this model provides a good fit to the data, there is expected to be a better correspondence between the bootstrap estimates and the standard estimates of $SE(\hat{\beta}_0)$, $SE(\hat{\beta}_1)$ and $SE(\hat{\beta}_2)$.

In [161]:
def coef2(X, Y):
    lm = sk_lm.LinearRegression()
    poly = PolynomialFeatures(degree=2)
    X_current = poly.fit_transform(X.values.reshape(-1,1))
    model = lm.fit(X_current, Y)
    return np.append(model.intercept_, model.coef_[1:])

In [166]:
boot(auto, coef2, 1000, x='horsepower', y='mpg')

Bootstrap Statistics:
Estimate: [ 5.69371890e+01 -4.66587462e-01  1.23128996e-03]
STD: [2.09911479e+00 3.32260259e-02 1.19470045e-04]



In [165]:
lm_fit = smf.ols('mpg~horsepower+np.power(horsepower,2)',
                data=auto).fit()
lm_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Fri, 08 Jan 2021   Prob (F-statistic):           5.40e-99
Time:                        23:54:59   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  56.9001      1.800     31.604      0.000      53.360      60.440
horsepower                 -0.4662      0.031    -14.978      0.000      -0.527      -0.405
np.power(horsepower, 2)     0.0012      0.000     10.080      0.000       0.001       0.001
==============================================================================
Omnibus:                       16.158   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.662
Skew:                           0.218   Prob(JB):                     2.20e-07
Kurtosis:                       4.299   Cond. No.                     1.29e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""